# Heatmaps
This file generates png-images and the GeoJSON-files used for heatmaps and fastest-transport mapping. 

# Imports, Methods & Data
These are automatically executed using the initialisation-cells plugin.

In [30]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from math import ceil
from pyproj import CRS

import seaborn as sns
sns.set()

import contextily as ctx
CTX_PROVIDER = ctx.providers.OpenStreetMap.Mapnik

from tqdm.notebook import tqdm
tqdm.pandas()

%matplotlib inline

DROOT = './data/'

In [31]:
# Used to translate the legendas on graphs to useful terms.
translate = {
    'walk_t': 'Walking',
    'car_r_t': 'Driving at rush-hour (8:00)',
    'car_m_t': 'Driving at mid-day (12:00)',
    'bike_t': 'Bicycling',
    'pt_r_t': 'Public transit at rush-hour (8:00)',
    'pt_m_t': 'Public transit at mid-day (12:00)'
}

route_cols = list(translate.keys())
route_cols_tl = list(translate.values())

In [32]:
def set_parktime(gdf_original, parking_time, humancols=False, minutes=False):
    """Returns a new copy of dataframe with the set parking_time."""

    # Check if a valid gdf_original
    modes_r = ['bike_t', 'car_r_t', 'pt_r_t', 'walk_t']
    modes_m = ['bike_t', 'car_m_t', 'pt_m_t', 'walk_t']

    required = [col in gdf_original.columns for col in (modes_r + modes_m)]
    if np.array(required).sum() != len(required):
        raise ValueError("Not all required columns provided.")

    # Build new copy of the original map, returning it later.
    new_gdf = gdf_original.copy()
    new_gdf.car_r_t = gdf_original.car_r_t + parking_time
    new_gdf.car_m_t = gdf_original.car_m_t + parking_time

    # Get fastest mode in rush hour and midday per cell.
    modes_r = ['bike_t', 'car_r_t', 'pt_r_t', 'walk_t']
    modes_m = ['bike_t', 'car_m_t', 'pt_m_t', 'walk_t']
    new_gdf['recmode_r'] = new_gdf[modes_r].idxmin(1)
    new_gdf['recmode_m'] = new_gdf[modes_m].idxmin(1)

    # Rename column values so we don't mix them up next.
    new_gdf.recmode_r.replace({'bike_t': 'bike_r_t',
                               'walk_t': 'walk_r_t'}, inplace=True)
    new_gdf.recmode_m.replace({'bike_t': 'bike_m_t',
                               'walk_t': 'walk_m_t'}, inplace=True)

    # If humancols is passed, we humanize the name of the cols.
    if humancols: 
        pass
    
    # If statistics are requested in minutes, convert them.
    if minutes:
        new_gdf[route_cols] = (new_gdf[route_cols] / 60).round(1)
    
    return new_gdf


In [33]:
# Load Cities transport times.
gdf_original = gpd.read_file(
    os.path.join(DROOT, '4-processed', 'complete-dataset.gpkg'))

# Create a copy to use later.
gdf_total = set_parktime(gdf_original, 0)

# Procedures

## Generate all city images for 7.5min waiting time

In [20]:
# Set waiting time to 7.5 minutes.
gdf_total = set_parktime(gdf_original, 450, minutes=True, humancols=True)

# Create directory
os.makedirs(os.path.join(DROOT, '5-graphs', 'heatmaps'), exist_ok=True)

# Loop over cities
iterator = tqdm(gdf_total[gdf_total.city ==
                          'Bern'].groupby('city'), leave=False)
for cityname, citydf in iterator:

    iterator.set_description(cityname)

    # Outpath for images
    outpath = os.path.join(DROOT, '5-graphs', 'heatmaps',
                           'heatmaps-%s-7.5min.png' % cityname)

    # Skip existing images (if not set to False)
    if os.path.exists(outpath) and True:
        continue

    # Generate subplots for all 6 graphs.
    fig, axs = plt.subplots(2, 3, figsize=(10, 6), dpi=150,
                            facecolor='w',
                            constrained_layout=True,
                            sharex=True, sharey=True,
                            subplot_kw=dict(aspect='equal'))
    axs = axs.ravel()

    for i in range(len(route_cols)):
        citydf.dropna(subset=[route_cols[i]]).plot(
            column=route_cols[i],
            linewidth=0,
            missing_kwds={'color': 'lightgrey',
                          "label": "Unavailable"},
            vmin=0, vmax=citydf.car_r_t.quantile(0.9),
            cmap='Spectral', alpha=0.8, ax=axs[i])
        ctx.add_basemap(axs[i], crs=citydf.crs,
                        source=CTX_PROVIDER, attribution=False)

        citydf[citydf.sky_d < 0.01].centroid.plot(ax=axs[i])
        axs[i].set_title(route_cols[i])
        axs[i].axis('off')

    patch_col = axs[0].collections[0]
    fig.colorbar(patch_col, ax=axs, shrink=0.5)

    fig.savefig(outpath)
    plt.close()

/Users/ivo/opt/anaconda3/envs/thesis-env/lib/python3.7/site-packages/geopandas/plotting.py:376: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

<Figure size 1500x900 with 6 Axes>

## Generate fastest transport mode images

In [ ]:
cmap = mpl.colors.ListedColormap([
    (0.996094, 0.500000, 0.113281),  # Orange
    (0.171875, 0.492188, 0.710938),  # Blue
    (0.269531, 0.683594, 0.320312),  # Green
    (0.601562, 0.304688, 0.625000)]) # Purple

In [ ]:
# Set waiting time to 7.5 minutes.
gdf_total = set_parktime(gdf_original, 450, humancols=True)

# Generate output directory.
os.makedirs(os.path.join(DROOT, '5-graphs', 'fastest-transport'), exist_ok=True)

# Create for ALL cities individually
iterator = tqdm(gdf_total.groupby('city'), leave=False)
for cityname, citydf_group in iterator:

    iterator.set_description(cityname)
    outpath = os.path.join(DROOT, '5-graphs', 'fastest-transport',
                             f'fastest-transport-{cityname}-7.5min.png')

    if os.path.exists(outpath) and True:
        continue
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 6), facecolor='w', dpi=200,
                            tight_layout=True, sharex=True, sharey=True)

    # TODO: add saturation by cell_pop as third variable in colors
    citydf.plot(column='recmode_r', legend=True,
                missing_kwds={'color': 'lightgrey', "label": "Unavailable"},
                linewidth=0, categorical=True,
                cmap=cmap, alpha=0.8, ax=axs[0])
    ctx.add_basemap(axs[0], crs=citydf.crs,
                    source=CTX_PROVIDER, attribution=False)
    citydf[citydf.sky_d < 0.01].centroid.plot(ax=axs[0])
    axs[0].set_title('Fastest mode in rush-hour (8:00)')
    axs[0].axis('off')

    # Plot
    citydf.plot(column='recmode_m', legend=True,
                missing_kwds={'color': 'lightgrey', "label": "Unavailable"},
                linewidth=0, categorical=True,
                cmap=cmap, alpha=0.8, ax=axs[1], )
    ctx.add_basemap(axs[1], crs=citydf.crs,
                    source=CTX_PROVIDER, attribution=False)
    citydf[citydf.sky_d < 0.01].centroid.plot(ax=axs[1])
    axs[1].set_title('Fastest mode at midday (12:00)')
    axs[1].axis('off')

    fig.savefig(outpath)
    plt.close()

## Generate GeoJSON for all cities

In [34]:
# Set waiting time to 0 minutes.
gdf_total = set_parktime(gdf_original, 0, minutes=True)

# Loop over cities
iterator = tqdm(gdf_total.groupby('city'), leave=False)
for cityname, citydf in iterator:
    
    iterator.set_description(cityname)
    
    # Outpath for imagesimport os
    os.makedirs( os.path.join(DROOT, '4-processed', 'heatmaps'), exist_ok=True)
    outpath = os.path.join(DROOT, '4-processed', 'heatmaps', 
                           cityname.replace(' ', '_') + '.geojson')
    
    if os.path.exists(outpath):
        continue
    
    citydf.to_file(outpath, driver="GeoJSON")


# Visualisations Paper

## Fastest Transport Mode

In [ ]:
# Create combined fastest transport image for 4 selected cities.
sel_cities = ['Amsterdam', 'London',  'Los Angeles', 'Sydney']

fig, axs = plt.subplots(2, 4, figsize=(16, 8), facecolor='w', dpi=150,
                        constrained_layout=True)

# Create images for 2.5 minutes waiting time and 7.5 minutes.
waiting = [0, 450]
for wt_i in range(len(waiting)):
    
    # Set the values to the ones we need here.
    gdf_total = set_parktime(gdf_original, waiting[wt_i])
    
    # Drop walk (too little), add at least 1 transit (keep mpl colors consistent)
    gdf_total.recmode_r = gdf_total.recmode_r.replace({'walk_r_t': 'bike_r_t'})
    gdf_total.loc[gdf_total.sky_d < 0.01, 'recmode_r'] = 'Transit'
    
    # Replace with friendly names.
    gdf_total.recmode_r = gdf_total.recmode_r.replace({
        'bike_r_t': 'Bicycling', 
        'pt_r_t': 'Transit', 
        'car_r_t': 'Driving'})
    
    selected = gdf_total.set_index('city').loc[sel_cities]

    # Loop over cities per row
    city_i = 0
    iterator = tqdm(selected.groupby('city'), leave=False)
    for cityname, citydf_grp in iterator:
        
        iterator.set_description(cityname + str(waiting[wt_i]))
        
        # Plot & create basemap        
        citydf_grp.plot(column='recmode_r', 
                        legend=(city_i == len(sel_cities) - 1 and wt_i == 0),
                        missing_kwds={'color': 'lightgrey', 
                                  "label": "Unavailable"},
                    linewidth=0, categorical=True,
                    cmap=cmap, alpha=0.7, ax=axs[wt_i, city_i])
        ctx.add_basemap(axs[wt_i, city_i], crs=citydf_grp.crs,
                        source=CTX_PROVIDER, attribution=False)
        citydf_grp[citydf_grp.sky_d < 0.01].centroid.plot(ax=axs[wt_i, city_i])
        axs[wt_i, city_i].axis('off')
        
        # Only set map title on first row.
        if wt_i == 0:
            axs[wt_i, city_i].set_title(f'Rush-hour in {cityname}')

        city_i += 1
    
fig.text(0.008, 0.25, 'no parking time', 
         ha='center',  va='center', rotation='vertical')

fig.text(0.008, 0.75, '7.5min parking time', 
         ha='center',  va='center', rotation='vertical')

fig.savefig(os.path.join(DROOT, '5-graphs',
                         'fastest-transport-samples.png'))

## Cover-image

In [ ]:
# Make sure to have the city to frame already in the citydf variable.
lat, lon = (52.3724, 4.9008)

height = 0.09
width = 0.5

west, south, east, north = (
    lon - width,
    lat - height,
    lon + width,
    lat + height,
)

ams_img, ams_ext = ctx.bounds2img(
    west, south, east, north, ll=True, 
    source=ctx.providers.Stamen.TerrainBackground
)

citydf_new = gdf_total[gdf_total.city == 'Amsterdam'].copy()
citydf_new.geometry = citydf_new.geometry.buffer(20)

fig, ax = plt.subplots(1, figsize=(20, 6))
ax.imshow(ams_img, extent=ams_ext)
citydf_new.plot(column='recmode_r', legend=True, linewidth=0, categorical=True,
                cmap='Set3', ax=ax)
citydf_new[citydf_new.sky_d < 0.01].centroid.plot(ax=ax, color='black')
ax.axis('off')

fig.savefig(os.path.join(DROOT, '5-graphs', 'cover-fastest-transport-%s-7.5min.png' % cityname))